#Description about the project
 This is Natural Language Processing project for E-commerce dataset in a json format with about 7 different classes of tags that could help user get fmailiar with the chatbot using pytorch CNN model Neural Net 



In [ ]:
#importing neccessary libraries
import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import numpy as np


USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [ ]:
#nlp pipleines 
nltk.download('punkt')  #a package with a pre-trained tokenized words
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower()) 

def bag_of_words(tokenized_sentence, all_word):
    tokenized_sentence = [stem(w) for w in tokenized_sentence]
    bag = np.zeros(len(all_word), dtype=np.float32)
    for idx, w in enumerate(all_word):
        if w in tokenized_sentence:
            bag[idx] = 1.0
    return bag

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#testing different nlp pipelines task on different examples 
a = 'See you later, thanks for visiting'
print(a)
b = tokenize(a)
print(b)

See you later, thanks for visiting
['See', 'you', 'later', ',', 'thanks', 'for', 'visiting']


In [ ]:
s = 'organizing', 'organization', 'organ'
print(s)
s=[stem(w) for w in s]
print(s)

('organizing', 'organization', 'organ')
['organ', 'organ', 'organ']


In [ ]:
sentence = ["hello", "how", "are", "you"]
words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
bg = bag_of_words(sentence, words)
print(bg)

[0. 1. 0. 1. 0. 0. 0.]


In [ ]:
#reading the json datataset file
import json
with open('intent.json', 'r') as f:
    intents = json.load(f)
#print(intents)
all_words = []
tags = []
patternresponse = []

In [ ]:
#loop through the tags (classes)
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        patternresponse.append((w,tag))

ignore_words = ['?', '!', '/', '#', '@', ',','.']

In [ ]:
#apply steeming (nlp) to the dataset
all_words = (stem(w) for w in all_words if w not in ignore_words)
all_words =sorted(set(all_words))
tags =sorted(set(tags))
print(tags)
print(all_words)

['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']
["'s", 'a', 'accept', 'anyon', 'are', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'deliveri', 'do', 'doe', 'funni', 'get', 'good', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'is', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'my', 'of', 'onli', 'pay', 'paypal', 'see', 'sell', 'ship', 'someth', 'take', 'tell', 'thank', 'that', 'there', 'what', 'when', 'which', 'with', 'you']


In [ ]:
#splitting the train dataset
X_train = []
y_train = []

for (pattern_sentence, tag) in patternresponse:
        bag = bag_of_words(pattern_sentence, all_words)
        X_train.append(bag)

        label = tags.index(tag)
        y_train.append(label)

X_train= np.array(X_train)
y_train = np.array(y_train)

In [ ]:
import torch.nn as nn

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
#creating a chat dataset 
class Chatdataset(Dataset):
    def __init__(self):
        self.n_sample = len(X_train)
        self.x_data = X_train
        self.y_data = y_train
#Dataset(idx)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_sample

In [ ]:
#hyperparameters for the cnn model
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

54 7


In [ ]:
dataset = Chatdataset()
train_dataset = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=2)
#running the model to cuda for quick runtime
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
#The model is a Feed Forward NN 2 hidden layers and 1 input layer'''
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)  
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)

        out = self.l1(x)
        out = self.relu(out)

        out = self.l2(out)
        out = self.relu(out)

        out = self.l3(out)
        #no relu or activation 

        return out 

In [ ]:
#making the model an object and passing arguments to run with on GPU or CPU platform
model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [ ]:
#loss and optimizer 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
correct = 0
total = 0
for epoch in range(num_epochs):
    for (words, labels) in train_dataset:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
    
        #backwards 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)


        if (epoch+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:2f}')


print(f'final loss: {loss.item():.4f}')
print(f'final accuracy: {accuracy:.2f}')

Epoch [100/1000], Loss: 0.8623, Accuracy: 100.000000
Epoch [100/1000], Loss: 1.1597, Accuracy: 100.000000
Epoch [100/1000], Loss: 0.9810, Accuracy: 100.000000
Epoch [100/1000], Loss: 0.6250, Accuracy: 100.000000
Epoch [200/1000], Loss: 0.1337, Accuracy: 100.000000
Epoch [200/1000], Loss: 0.1152, Accuracy: 100.000000
Epoch [200/1000], Loss: 0.2386, Accuracy: 100.000000
Epoch [200/1000], Loss: 0.0712, Accuracy: 100.000000
Epoch [300/1000], Loss: 0.0381, Accuracy: 100.000000
Epoch [300/1000], Loss: 0.0189, Accuracy: 100.000000
Epoch [300/1000], Loss: 0.0547, Accuracy: 100.000000
Epoch [300/1000], Loss: 0.0241, Accuracy: 100.000000
Epoch [400/1000], Loss: 0.0084, Accuracy: 100.000000
Epoch [400/1000], Loss: 0.0181, Accuracy: 100.000000
Epoch [400/1000], Loss: 0.0134, Accuracy: 100.000000
Epoch [400/1000], Loss: 0.0209, Accuracy: 100.000000
Epoch [500/1000], Loss: 0.0093, Accuracy: 100.000000
Epoch [500/1000], Loss: 0.0055, Accuracy: 100.000000
Epoch [500/1000], Loss: 0.0052, Accuracy: 100.

In [ ]:
data = {
    "model_state":model.state_dict(),
    "input_size":input_size,
    "output_size":output_size,
    "hidden_size":hidden_size,
    "all_words": all_words,
    "tags": tags
}

FILE = 'data.pth' #extension to save file in pytorch
torch.save(data,FILE)

print("training completed, save to {FILE}")

training completed, save to {FILE}


In [ ]:
#building a non-AI run code for the NN model built 
with open('/content/intent.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "Jude"
print("Let's chat! (type 'quit' to exit)")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]

    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")

Let's chat! (type 'quit' to exit)
You: hello
Israel: Hello, thanks for visiting
You: what do you sell
Israel: We sell coffee and tea
You: what is the cost
Israel: Hello, thanks for visiting
You: what is the price
Israel: Hello, thanks for visiting
You: what is the delivery 
Israel: I do not understand...
You: how do you make payment 
Israel: Hi there, what can I do for you?
You: how to pay
Israel: Hi there, how can I help?
You: do take cash
Israel: We accept most major credit cards, and Paypal
You: how about mode of delivery
Israel: I do not understand...
You: deliver goods
Israel: Hello, thanks for visiting
You: quit
